In [20]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

# Load MNIST dataset
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()

# Preprocess the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = x_train.reshape((x_train.shape[0], np.prod(x_train.shape[1:])))
x_test = x_test.reshape((x_test.shape[0], np.prod(x_test.shape[1:])))

# Define the VAE architecture
original_dim = 784
latent_dim = 2

# Encoder network
input_img = Input(shape=(original_dim,))
hidden_layer = Dense(256, activation='relu')(input_img)
z_mean = Dense(latent_dim)(hidden_layer)
z_log_var = Dense(latent_dim)(hidden_layer)

# Custom sampling layer
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.random.normal(shape=(tf.shape(z_mean)[0], latent_dim), mean=0., stddev=1.0)
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling)([z_mean, z_log_var])

# Decoder network
decoder_hidden = Dense(256, activation='relu')(z)
output_img = Dense(original_dim, activation='sigmoid')(decoder_hidden)

# Create VAE model
vae = Model(input_img, output_img)

# Define loss function
def vae_loss(x, x_decoded_mean):
    xent_loss = original_dim * tf.keras.losses.binary_crossentropy(x, x_decoded_mean)
    kl_loss = -0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1)
    return xent_loss + kl_loss

# Compile the VAE
vae.compile(optimizer='adam', loss=vae_loss)

# Train the VAE
epochs = 50
batch_size = 128
vae.fit(x_train, x_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, x_test))

# Generate a grid of digits using the trained VAE decoder
n = 15  # Number of digits to display
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))

# Linearly spaced coordinates on the unit square were transformed through the inverse CDF (ppf) of the Gaussian
# to produce values of the latent variables z, since the prior of the latent space is Gaussian
grid_x = np.linspace(-4, 4, n)
grid_y = np.linspace(-4, 4, n)[::-1]

for i, yi in enumerate(grid_y):
    for j, xi in enumerate(grid_x):
        z_sample = np.array([[xi, yi]])
        x_decoded = vae.decoder.predict(z_sample)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
               j * digit_size: (j + 1) * digit_size] = digit

# Save the generated images
plt.figure(figsize=(10, 10))
plt.imshow(figure, cmap='Greys_r')
plt.axis('off')
plt.savefig('results/vae/generated_digits.png')
plt.show()

Epoch 1/50


2023-08-12 22:20:14.643381: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'tf.range_25/range/vae_loss/weighted_loss/Const_2' with dtype int32
	 [[{{node tf.range_25/range/vae_loss/weighted_loss/Const_2}}]]


TypeError: in user code:

    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1268, in step_function  **
        
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in run_step  **
        
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_step
        
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1109, in compute_loss
        
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/compile_utils.py", line 317, in __call__
        
    File "/opt/conda/lib/python3.10/site-packages/keras/utils/metrics_utils.py", line 77, in decorated
        
    File "/opt/conda/lib/python3.10/site-packages/keras/metrics/base_metric.py", line 140, in update_state_fn
        
    File "/opt/conda/